# Mapping the socio-economic situation in Mpumalanga, South Africa

In order to meet its commitments to the Paris agreements, South Africa is investigating ways to ween itself off of coal. This will be felt especially strongly in Mpumalanga, South Africa, a province in the north-east of the country between Johannesburg and Eswatini. Mpumalanga is a sparsely populated area home to most of South Africa's coal communities and coal power plants.

In [1]:
import sys
import os
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

from shapely.geometry import box, Point

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTRocks.dataMisc as dataMisc
import GOSTRocks.ntlMisc as ntlMisc
import GOSTRocks.ghslMisc as ghslMisc
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.insert(0, "/home/wb411133/Code/GOSTNets_Raster/src")
import GOSTNetsRaster.market_access as ma

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Define input variables
in_folder = "/home/wb411133/projects/URB_SURDR_ZAF"
ntl_folder = os.path.join(in_folder, "NTL_data")
ghsl_folder = os.path.join(in_folder, "GHSL_data")
urban_folder = os.path.join(in_folder, "URBAN")
ma_folder = os.path.join(in_folder, "market_access")
infra_folder = os.path.join(in_folder, "Infra")
protected_areas_folder = os.path.join(in_folder, "Protected_Areas")
for f in [
    in_folder,
    ntl_folder,
    ghsl_folder,
    ma_folder,
    infra_folder,
    protected_areas_folder,
]:
    if not os.path.exists(f):
        os.makedirs(f)

# Define global variables
global_bounds = (
    "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM0.shp"
)
ghs_folder = "/home/public/Data/GLOBAL/GHSL"
ghs_built_folder = os.path.join(ghs_folder, "Built")
ghs_built_files = [x for x in os.listdir(ghs_built_folder) if x.endswith(".tif")]
ghs_smod_file = os.path.join(
    ghs_folder, "SMOD", "GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
)
ghs_ucdb = os.path.join(
    ghs_folder, "GHS_UCBD_R2019A", "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
)
global_friction = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_airports_file = os.path.join(
    infra_folder, "airport_volume_airport_locations.csv"
)

# Define local variables
admin0_file = os.path.join(in_folder, "ZAF_select_adm0.shp")
admin3_file = os.path.join(in_folder, "ADMIN", "admin3_geoBounds_FINAL.shp")
ghsl_thresh = 0.1
local_ghsl_file = os.path.join(in_folder, f"ghsl_combined_{int(ghsl_thresh*100)}.tif")
urban_raster = os.path.join(urban_folder, "zaf1k_cpo20_urban.tif")
urban_raster_pop = os.path.join(urban_folder, "zaf1k_cpo20.tif")
urban_extents_file = os.path.join(urban_folder, "cpo20_urban_extents.shp")
local_ghs_smod_file = os.path.join(in_folder, "GHS_SMOD_2020.tif")
major_urban_extents = os.path.join(in_folder, "major_cities_UCDB2019.shp")
local_friction_file = os.path.join(ma_folder, "friction_2020.tif")
local_airports = os.path.join(infra_folder, "airports_ZAF_and_neighbours.geojson")
local_ports = os.path.join(infra_folder, "Ports.shp")
tourist_locations = os.path.join(infra_folder, "Kruger_EntryGates.shp")
protected_areas = os.path.join(protected_areas_folder, "SAPAD_IR_2023_Q2_01.shp")

In [3]:
# read in base data
ntl_files = dataMisc.aws_search_ntl()
admin0_bounds = gpd.read_file(global_bounds)
if not os.path.exists(admin0_file):
    zaf_bounds = admin0_bounds.loc[admin0_bounds["WB_A3"] == "ZAF"]
    zaf_bounds.to_file(admin0_file)
else:
    zaf_bounds = gpd.read_file(admin0_file)
neighbours = admin0_bounds.loc[
    admin0_bounds.intersects(zaf_bounds.unary_union.buffer(0.1))
]
#
admin1_bounds = dataMisc.get_geoboundaries("ZAF", "ADM1")
admin2_bounds = dataMisc.get_geoboundaries("ZAF", "ADM2")
admin3_bounds = dataMisc.get_geoboundaries("ZAF", "ADM3")
focal_state = admin1_bounds.loc[admin1_bounds["shapeName"] == "Mpumalanga"]

In [ ]:
admin1_bounds.to_file(os.path.join(in_folder, "admin1_geoBounds.shp"))
admin2_bounds.to_file(os.path.join(in_folder, "admin2_geoBounds.shp"))
admin3_bounds.to_file(os.path.join(in_folder, "admin3_geoBounds.shp"))

# Clip raster datasets

In [ ]:
# Clip out nighttime lights annual images
# Mpumalanga
ntlMisc.generate_annual_composites(
    focal_state.unary_union, out_folder=os.path.join(ntl_folder, "Mpumalanga")
)
# ZAF
# ntlMisc.generate_annual_composites(zaf_bounds.unary_union, out_folder=os.path.join(ntl_folder, "ZAF"))
# Neighbours
# ntlMisc.generate_annual_composites(neighbours.unary_union, out_folder=os.path.join(ntl_folder, "Neighbours"))

In [ ]:
# Clip out GHSL layers
for cur_raster_file in ghs_built_files:
    out_file = os.path.join(ghsl_folder, os.path.basename(cur_raster_file))
    if not os.path.exists(out_file):
        rMisc.clipRaster(
            rasterio.open(os.path.join(ghs_built_folder, cur_raster_file)),
            zaf_bounds,
            out_file,
            crop=False,
        )
        tPrint(out_file)

# Combine GHSL layers into single file
ghsl_files = sorted(
    [
        os.path.join(ghsl_folder, x)
        for x in os.listdir(ghsl_folder)
        if x.endswith(".tif")
    ]
)
if not os.path.exists(local_ghsl_file):
    ghsl_res = ghslMisc.combine_ghsl_annual(
        ghsl_files, built_thresh=ghsl_thresh, out_file=out_file
    )

In [ ]:
# clip out GHS-SMOD data
if not os.path.exists(local_ghs_smod_file):
    rMisc.clipRaster(
        rasterio.open(ghs_smod_file), neighbours, local_ghs_smod_file, crop=False
    )

In [ ]:
# Convert urban centres from the constrained world_pop 2020 dataset to vector
if not os.path.exists(urban_extents_file):
    urban_extents = rMisc.vectorize_raster(rasterio.open(urban_raster), bad_vals=[0])
    urban_extents["geometry"] = urban_extents["geometry"].apply(lambda x: x.buffer(0))

    # Attribute with population
    res = rMisc.zonalStats(urban_extents, urban_raster_pop, minVal=0)
    res = pd.DataFrame(res, columns=["SUM", "MIN", "MAX", "MEAN"])
    urban_extents["Pop2020"] = res["SUM"]
    urban_extents.to_file(urban_extents_file)

In [ ]:
# Extract major settlements from UCDB
if not os.path.exists(major_urban_extents):
    all_extents = gpd.read_file(ghs_ucdb)
    sel_extents = all_extents.loc[all_extents.intersects(box(*neighbours.total_bounds))]
    sel_extents.to_file(major_urban_extents)

In [ ]:
all_extents.head()

# Extract international airports

In [ ]:
if not os.path.exists(local_airports):
    inA = pd.read_csv(global_airports_file)
    inA_geom = [
        Point(x) for x in zip(inA["Airport1Longitude"], inA["Airport1Latitude"])
    ]
    inA = gpd.GeoDataFrame(inA, geometry=inA_geom, crs=4326)
    selA = inA.loc[inA.intersects(neighbours.unary_union)]
    selA.to_file(local_airports, driver="GeoJSON")

"""headers = {'Accept': 'application/json'}
ddh_international_airports = "https://wiki.worldbank.org/pages/viewpage.action?spaceKey=GEOS&title=Guide+to+procurement+of+satellite+imagery+and+derived+products"
ddh_r = requests.get(ddh_international_airports, headers=headers)
ddh_r.json()"""

# Market access

In [ ]:
if not os.path.exists(local_friction_file):
    rMisc.clipRaster(
        rasterio.open(global_friction), neighbours, local_friction_file, crop=False
    )

friction = rasterio.open(local_friction_file)
frictionD = friction.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
# Measure access to all major cities in ucdb
tt_major_cities = os.path.join(ma_folder, "tt_major_cities.tif")
if not os.path.exists(tt_major_cities):
    dests = gpd.read_file(major_urban_extents)
    dests["geometry"] = dests["geometry"].apply(lambda x: x.centroid)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, dests)
    with rasterio.open(tt_major_cities, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to international airports
tt_airports = os.path.join(ma_folder, "tt_airports.tif")
if not os.path.exists(tt_airports):
    airports = gpd.read_file(local_airports)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, airports)
    with rasterio.open(tt_airports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to international ports
tt_ports = os.path.join(ma_folder, "tt_ports.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(local_ports)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to kruger national park
tt_ports = os.path.join(ma_folder, "tt_tourism.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(tourist_locations)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to all national protected areas
tt_ports = os.path.join(ma_folder, "tt_protected.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(protected_areas)
    ports["geometry"] = ports["geometry"].apply(lambda x: x.centroid)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
dests = gpd.read_file(admin3_file)
dests["geometry"] = dests["geometry"].apply(lambda x: x.centroid)

od_res = ma.calculate_od_matrix(friction, mcp, dests)
final_od = pd.DataFrame(od_res)
final_od.to_csv(os.path.join(ma_folder, "admins3_od.csv"))

In [ ]:
# calculate total population and nighttime lights brightness for each admin3
dests = gpd.read_file(admin3_file)
pop_res = rMisc.zonalStats(dests, urban_raster_pop, minVal=0, reProj=True)
pop_res = pd.DataFrame(pop_res, columns=["SUM", "MIN", "MAX", "MEAN"])
pop_res

In [ ]:
ntl_raster = os.path.join(ntl_folder, "Neighbours", "VIIRS_2022_annual.tif")
ntl_res = rMisc.zonalStats(dests, ntl_raster, minVal=1, reProj=True)
ntl_res = pd.DataFrame(ntl_res, columns=["SUM", "MIN", "MAX", "MEAN"])

In [ ]:
# Map random
dests["Pop"] = pop_res["SUM"]
dests["NTL"] = pop_res["SUM"]
mapMisc.static_map_vector(
    dests,
    "Pop",
    legend_loc="upper left",
    thresh=[0, 50000, 100000, 250000, 500000, 200000000],
)

# Calculate Main Place to Protected Areas OD

In [11]:
municipalities = os.path.join(
    in_folder, "MiningCommunities", "MainPlaces", "MP_SA_2011.shp"
)
inM = gpd.read_file(municipalities)
inM["geometry"] = inM["geometry"].apply(lambda x: x.centroid)

destinations = gpd.read_file(protected_areas)
destinations = destinations.to_crs(22293)
destinations["area_km"] = destinations["geometry"].apply(lambda x: x.area / 1000000)
destinations = destinations.to_crs(4326)
destinations["geometry"] = destinations["geometry"].apply(lambda x: x.centroid)

In [12]:
# Calculate travel time
popR = rasterio.open(urban_raster_pop)
ttr = rasterio.open(local_friction_file)
frictionD = ttr.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [13]:
od = ma.calculate_od_matrix(ttr, mcp, inM, destinations)
xx = pd.DataFrame(od)
xx.columns = inM.MP_CODE_st
xx.index = destinations.WDPAID
xx.to_csv(os.path.join(out_varun_folder, "mp_protected_areas_OD.csv"))

09:51:23	Starting 0
09:51:30	Starting 1
09:51:37	Starting 2
09:51:44	Starting 3
09:51:51	Starting 4
09:51:58	Starting 5
09:52:05	Starting 6
09:52:12	Starting 7
09:52:19	Starting 8
09:52:26	Starting 9
09:52:33	Starting 10
09:52:40	Starting 11
09:52:47	Starting 12
09:52:54	Starting 13
09:53:01	Starting 14
09:53:08	Starting 15
09:53:15	Starting 16
09:53:22	Starting 17
09:53:29	Starting 18
09:53:36	Starting 19
09:53:43	Starting 20
09:53:49	Starting 21
09:53:56	Starting 22
09:54:03	Starting 23
09:54:10	Starting 24
09:54:17	Starting 25
09:54:24	Starting 26
09:54:31	Starting 27
09:54:39	Starting 28
09:54:46	Starting 29
09:54:53	Starting 30
09:55:00	Starting 31
09:55:07	Starting 32
09:55:14	Starting 33
09:55:21	Starting 34
09:55:28	Starting 35
09:55:35	Starting 36
09:55:41	Starting 37
09:55:48	Starting 38
09:55:55	Starting 39
09:56:02	Starting 40
09:56:09	Starting 41
09:56:16	Starting 42
09:56:23	Starting 43
09:56:30	Starting 44
09:56:37	Starting 45
09:56:44	Starting 46
09:56:51	Starting 47
09

10:34:17	Starting 378
10:34:24	Starting 379
10:34:31	Starting 380
10:34:38	Starting 381
10:34:45	Starting 382
10:34:53	Starting 383
10:35:02	Starting 384
10:35:11	Starting 385
10:35:19	Starting 386
10:35:28	Starting 387
10:35:37	Starting 388
10:35:45	Starting 389
10:35:54	Starting 390
10:36:02	Starting 391
10:36:10	Starting 392
10:36:18	Starting 393
10:36:26	Starting 394
10:36:35	Starting 395
10:36:43	Starting 396
10:36:51	Starting 397
10:36:59	Starting 398
10:37:08	Starting 399
10:37:16	Starting 400
10:37:25	Starting 401
10:37:33	Starting 402
10:37:42	Starting 403
10:37:50	Starting 404
10:37:58	Starting 405
10:38:07	Starting 406
10:38:15	Starting 407
10:38:24	Starting 408
10:38:33	Starting 409
10:38:41	Starting 410
10:38:49	Starting 411
10:38:57	Starting 412
10:39:06	Starting 413
10:39:15	Starting 414
10:39:23	Starting 415
10:39:32	Starting 416
10:39:41	Starting 417
10:39:49	Starting 418
10:39:58	Starting 419
10:40:07	Starting 420
10:40:17	Starting 421
10:40:26	Starting 422
10:40:35	S

11:18:39	Starting 751
11:18:45	Starting 752
11:18:52	Starting 753
11:18:59	Starting 754
11:19:06	Starting 755
11:19:13	Starting 756
11:19:19	Starting 757
11:19:26	Starting 758
11:19:33	Starting 759
11:19:40	Starting 760
11:19:47	Starting 761
11:19:54	Starting 762
11:20:00	Starting 763
11:20:07	Starting 764
11:20:14	Starting 765
11:20:21	Starting 766
11:20:28	Starting 767
11:20:34	Starting 768
11:20:41	Starting 769
11:20:48	Starting 770
11:20:55	Starting 771
11:21:02	Starting 772
11:21:09	Starting 773
11:21:15	Starting 774
11:21:22	Starting 775
11:21:29	Starting 776
11:21:36	Starting 777
11:21:43	Starting 778
11:21:50	Starting 779
11:21:56	Starting 780
11:22:03	Starting 781
11:22:10	Starting 782
11:22:17	Starting 783
11:22:24	Starting 784
11:22:31	Starting 785
11:22:37	Starting 786
11:22:44	Starting 787
11:22:51	Starting 788
11:22:58	Starting 789
11:23:04	Starting 790
11:23:11	Starting 791
11:23:18	Starting 792
11:23:25	Starting 793
11:23:32	Starting 794
11:23:38	Starting 795
11:23:45	S

12:00:14	Starting 1119
12:00:21	Starting 1120
12:00:28	Starting 1121
12:00:35	Starting 1122
12:00:41	Starting 1123
12:00:48	Starting 1124
12:00:55	Starting 1125
12:01:02	Starting 1126
12:01:09	Starting 1127
12:01:15	Starting 1128
12:01:22	Starting 1129
12:01:29	Starting 1130
12:01:36	Starting 1131
12:01:43	Starting 1132
12:01:50	Starting 1133
12:01:57	Starting 1134
12:02:03	Starting 1135
12:02:10	Starting 1136
12:02:17	Starting 1137
12:02:24	Starting 1138
12:02:30	Starting 1139
12:02:37	Starting 1140
12:02:44	Starting 1141
12:02:51	Starting 1142
12:02:58	Starting 1143
12:03:04	Starting 1144
12:03:11	Starting 1145
12:03:18	Starting 1146
12:03:25	Starting 1147
12:03:32	Starting 1148
12:03:39	Starting 1149
12:03:46	Starting 1150
12:03:53	Starting 1151
12:03:59	Starting 1152
12:04:06	Starting 1153
12:04:13	Starting 1154
12:04:20	Starting 1155
12:04:27	Starting 1156
12:04:33	Starting 1157
12:04:40	Starting 1158
12:04:47	Starting 1159
12:04:54	Starting 1160
12:05:01	Starting 1161
12:05:07	St

12:40:27	Starting 1476
12:40:34	Starting 1477
12:40:41	Starting 1478
12:40:48	Starting 1479
12:40:55	Starting 1480
12:41:02	Starting 1481
12:41:08	Starting 1482
12:41:15	Starting 1483
12:41:22	Starting 1484
12:41:29	Starting 1485
12:41:35	Starting 1486
12:41:42	Starting 1487
12:41:49	Starting 1488
12:41:56	Starting 1489
12:42:02	Starting 1490
12:42:09	Starting 1491
12:42:16	Starting 1492
12:42:23	Starting 1493
12:42:29	Starting 1494
12:42:36	Starting 1495
12:42:43	Starting 1496
12:42:50	Starting 1497
12:42:57	Starting 1498
12:43:04	Starting 1499
12:43:11	Starting 1500
12:43:17	Starting 1501
12:43:24	Starting 1502
12:43:31	Starting 1503
12:43:38	Starting 1504
12:43:45	Starting 1505
12:43:52	Starting 1506
12:43:59	Starting 1507
12:44:05	Starting 1508
12:44:12	Starting 1509
12:44:19	Starting 1510
12:44:26	Starting 1511
12:44:33	Starting 1512
12:44:40	Starting 1513
12:44:46	Starting 1514
12:44:53	Starting 1515
12:45:00	Starting 1516
12:45:07	Starting 1517
12:45:13	Starting 1518
12:45:20	St

ValueError: Length mismatch: Expected axis has 1651 elements, new values have 14039 elements

## Calculate Main Place OD

In [ ]:
municipalities = os.path.join(
    in_folder, "MiningCommunities", "MainPlaces", "MP_SA_2011.shp"
)
inM = gpd.read_file(municipalities)
inM["geometry"] = inM["geometry"].apply(lambda x: x.centroid)

In [ ]:
# Calculate travel time
popR = rasterio.open(urban_raster_pop)
ttr = rasterio.open(local_friction_file)
frictionD = ttr.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
od = ma.calculate_od_matrix(ttr, mcp, inM, inM)
xx = pd.DataFrame(od)
xx.columns = inM.MP_CODE_st
xx.index = sel_sp.MP_CODE_st
xx.to_csv(os.path.join(out_varun_folder, "all_mp.csv"))

In [ ]:
xx.columns = inM.MP_CODE_st
xx.index = inM.MP_CODE_st

In [ ]:
xx.to_csv(os.path.join(ma_folder, "all_mp_od.csv"))

In [ ]:
xx.head()

# Calculate gravity

In [ ]:
ma.calculate_gravity?

In [ ]:
weights = pd.read_csv(os.path.join(in_folder, "ZONAL_RES", "named_places_zonal.csv"))
weights.head()

In [ ]:
weights.shape

In [ ]:
xx.shape

In [ ]:
simple_gravity = ma.calculate_gravity(xx)
simple_gravity.head()

In [ ]:
pop_gravity = ma.calculate_gravity(xx, weights["POP"].values, weights["POP"].values)
pop_gravity.head()

In [ ]:
ntl_gravity = ma.calculate_gravity(
    xx, weights["NTL_2023"].values, weights["NTL_2023"].values
)
ntl_gravity.head()

In [ ]:
inM = gpd.read_file(municipalities)

In [ ]:
def create_ma_geometry(ma_df, out_file, xx_inM, ma_col=["d_0.001"], driver="GeoJSON"):
    # create output geospatial market access data
    simple_geog = ma_df.copy()
    simple_geog = simple_geog.loc[:, ma_col]
    simple_geog["geometry"] = xx_inM["geometry"].values
    simple_geog = gpd.GeoDataFrame(simple_geog, geometry="geometry", crs=xx_inM.crs)
    pd.DataFrame(simple_geog.drop(["geometry"], axis=1)).to_csv(f"{out_file}.csv")
    simple_geog.to_file(out_file, driver=driver)
    return simple_geog

In [ ]:
ma_folder

In [ ]:
create_ma_geometry(
    simple_gravity,
    os.path.join(ma_folder, "simple_ma.shp"),
    inM,
    driver="ESRI Shapefile",
)
create_ma_geometry(
    pop_gravity, os.path.join(ma_folder, "pop_ma.shp"), inM, driver="ESRI Shapefile"
)
create_ma_geometry(
    ntl_gravity, os.path.join(ma_folder, "ntl_ma.shp"), inM, driver="ESRI Shapefile"
)

In [ ]:
weights.head()

In [ ]:
weights.loc[weights["MP_CODE_st"] == 798020]

# Debugging